# Demo for CLIFuzz

## What is CLIFuzz?

`CLIFuzz` is a tool to fuzz command-line interface (CLI) utilities using their configuration options and arguments. It first automatically extracts the configuration options and arguments from CLI utilities which use `getopt`, `getopt_long` or `getopt_long_only` to parse its CLI invocations. It then constructs a JSON grammar representing its invocation language which is then used to grammar fuzz the utility.

Step 2. Consider an example CLI utility `cat` (v9.0) on the Linux command line that uses `getopt_long` to parse its invocations.

Step 3. We observe that `cat` can have multiple options in its invocation e.g., `-A`, `--show-ends` etc.

In [1]:
cat --help

Usage: cat [OPTION]... [FILE]...
Concatenate FILE(s) to standard output.

With no FILE, or when FILE is -, read standard input.

  -A, --show-all           equivalent to -vET
  -b, --number-nonblank    number nonempty output lines, overrides -n
  -e                       equivalent to -vE
  -E, --show-ends          display $ at end of each line
  -n, --number             number all output lines
  -s, --squeeze-blank      suppress repeated empty output lines
  -t                       equivalent to -vT
  -T, --show-tabs          display TAB characters as ^I
  -u                       (ignored)
  -v, --show-nonprinting   use ^ and M- notation, except for LFD and TAB
      --help     display this help and exit
      --version  output version information and exit

Examples:
  cat f - g  Output f's contents, then standard input, then g's contents.
  cat        Copy standard input to standard output.

GNU coreutils online help: <https://www.gnu.org/software/coreutils/>
Report cat translation

Step 4, 5. When we fuzz `cat` without incorporating its options in the fuzzing invocation, we miss out on a lot of code segments which can only be triggered by these options. A trivial example in `cat` is the code parsing its invocation:

    while ((c = getopt_long (argc, argv, "benstuvAET", long_options, NULL)) != -1) {
        switch (c) {
            ...
            case 'E':
                show_ends = true;
                break;
            ...
        }

The configuration options activate some flags/variables in this code, e.g., setting the `E` option in `cat` sets the `show_ends` flag to `true`. This flag in turn activates some code segment in the code. E.g., the value of the `show_ends` variable dictates the execution of the following code segment, among others, in `cat`. Excluding the `-E` option from the fuzzing invocations will never execute such code segments.

    if (show_ends)
      {
        if (pending_cr)
          {
            *bpout++ = '^';
            *bpout++ = 'M';
            pending_cr = false;
          }
        *bpout++ = '$';
      }      

Step 6. The table below lists the difference in code coverage between fuzzing `cat` without options using `AFL++` and fuzzing `cat` with options using `CLIFuzz`. We see that `CLIFuzz` is able to achieve better Line and Branch Coverage.

| Utility |#lines |#branches| AFL++ Line Coverage | AFL++ Branch Coverage | CLIFuzz Line Coverage | CLIFuzz Branch Coverage |
|---|---|---|---|---|---|---|
| `cat` (v9.0) |1044|784|33.94|22.71|35.81|25.56| 

Step 7. -> "We show 5--6 slides illustrating how \clifuzz works". Not sure what exactly to show here. - AG

Step 8. The script `run-cf-cmd.py` is the main script. It extracts the options and generates grammars of utilities. It also runs the fuzzer on the utilities using their grammars. Its `help` output describes its usage -

    ubuntu@clifuzzer:~/fse2022-clifuzzer/src$ python3 run-cf-cmd.py --help
    usage: run-cf-cmd.py [-h] [--get-grammar | --get-options | -f FUZZ | --get-coverage | --fuzz-coverage FUZZ_COVERAGE | --get-manual-coverage] [-o O] [-g GRAM_FILE] [--log-pass] [--invalid-options] [--invalid-values] [--seed SEED] binary 

    Run Configuration fuzzing on the given binary

    positional arguments: 
      binary                Binary or path to Binary to fuzz on

    optional arguments:
      -h, --help            show this help message and exit
      --get-grammar         Print the grammar extracted from options
      --get-options         Print a list of cmdline options available in the binary
      -f FUZZ, --fuzz FUZZ  No. of times the fuzzer should fuzz the binary and note unexpected behaviour including crashes and interesting return codes
      --get-coverage        Extract the coverage achieved in the tool
      --fuzz-coverage FUZZ_COVERAGE
                            Run fuzz FUZZ_COVERAGE times and then report the extracted coverage. Equivalent to running -f FUZZ_COVERAGE first and then running --get-coverage on a binary
      --get-manual-coverage 
                            Extract the manual testing coverage of the binary 
      -o O                  File to dump the output to. Defaults to binary-name.out
      -g GRAM_FILE, --gram-file GRAM_FILE
                            File containing the grammar to fuzz from.
      --log-pass            logs passing invocations as well.
      --invalid-options     Inserts invalid options into the grammar
      --invalid-values      Inserts invalid values for some options in the grammar
      --seed SEED           Seed for randomising    

Step 9 By invoking `run-cf-cmd.py` on `cat`, we can extract its options and the resultant grammar.


## Replicating our results

For easy replication of our results, you can follow the following steps -

Install `LXD` on the host machine, and in case of Linux based hosts, update `/proc/sys/kernel/core_pattern` as follows:

    # echo /tmp/core.%e.%p.%t > /proc/sys/kernel/core_pattern
    $ cat /proc/sys/kernel/core_pattern
    /tmp/core.%e.%p.%t